# Convolutional Neural Networks

<img src="mnist_sample.png", height=200, width=200>

## Task 1: Load the Data

Load the entire MNIST set from Tensorflow's dataset.

In [1]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# set batchSz batch gradient descent
batchSz = 50

# set hiddenSz for the hidden layer
hiddenSz = 1000 

# set learning rate 
l = 0.0004 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Placeholders:

In [2]:
# Images and their corresponding labels
img = tf.placeholder(tf.float32, [None,28,28,1])
ans = tf.placeholder(tf.float32, [None, 10])

## Architecture
* First convolution layer with 5x5 filter with 1 in channel and 32 in depth and SAME padding
* ReLU
* Maxpooling layer with 2x2 filter and SAME padding
* Second Convolution layer with 5x5 filter 32 in channel and 64 in depth
* ReLU
* Maxpooling layer with 2x2 filter and SAME padding
* Feed-forward layer from 3136 (you should have 3136 parameters at this point) to 1000
* ReLU
* Feed-forward layer from 1000 to 10
* Softmax/Sigmoid


In [4]:
# First Convolution layer with 5x5 filters, 1 in channel, and 32 depth 
conv1 = tf.layers.conv2d(inputs=img, 
                         filters=32, 
                         kernel_size=[5, 5], 
                         padding="same", 
                         activation=tf.nn.relu)

# Maxpooling layer with 2x2 filter and SAME padding
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, padding="same")

# Second convolution layer with 5x5 filters, 32 in channel, and 64 in depth
conv2 = tf.layers.conv2d(inputs=pool1, 
                         filters=64, 
                         kernel_size=[5, 5], 
                         padding="same", 
                         activation=tf.nn.relu)

# Maxpooling layer with 2x2 filter and SAME padding
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, padding="same")

#Feed-forward layer from 3136 (you should have 3136 parameters at this point) to 1000
pool2_flat = tf.reshape(pool2, [-1, 64*7*7])

# first hidden layer

fc1 = tf.layers.dense(pool2_flat, hiddenSz)

# the final layer

y_conv = tf.layers.dense(fc1, 10)
y_conv_sm = tf.nn.softmax(y_conv)

## Forward and Backward Pass
Write the forward and backward pass in tensorflow
* Use cross entropy loss
* Use AdamOptimizer

In [5]:
# Forward passs -- compute the cross entropy and output probabilities
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels = ans,
    logits = y_conv, # revise
    )

# Backward pass
# Note: you don't need to compute the gradient 
# Simply use the tf.train.AdamOptimizer() function
train = tf.train.AdamOptimizer(l).minimize(cross_entropy)



# Compute Accuracy
correct_prediction = tf.equal(tf.argmax(y_conv_sm, 1), tf.argmax(ans, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())


## Training the Network and Testing our Results
Train on 2000 batches

In [ ]:
start = time.time()

print("Training on 2000 batches:") 
step = 100
for i in range(2000):
    if i % step == 0: sstart = time.time()  
    imgs, anss = mnist.train.next_batch(batchSz)
    sess.run(train, feed_dict={img: imgs.reshape([batchSz,28,28,1]), ans: anss})
    if (i+1) % step == 0:
        sstop = time.time()
        print('batches %s - %d, duration ~ %g sec' % (i-(step-2), i+1, (sstop - sstart)))

# Batch size for evaluation performance
batchSz = 100
        
# Accuracy for train data
trainAcc = 0
batchNum =  int(mnist.train.num_examples/batchSz)
for i in range(batchNum):
    imgs, anss= mnist.train.next_batch(batchSz)
    trainAcc += sess.run(accuracy, feed_dict={img: imgs.reshape([batchSz,28,28,1]), ans: anss})
print ("Train Accuracy: %r" % (trainAcc/batchNum))

# Accuracy for test data
testAcc = 0
batchNum = int(mnist.test.num_examples/batchSz)
for i in range(batchNum):
    imgs, anss= mnist.test.next_batch(batchSz)
    testAcc += sess.run(accuracy, feed_dict={img: imgs.reshape([batchSz,28,28,1]), ans: anss})
print ("Test Accuracy: %r" % (testAcc/batchNum))

end = time.time()
print("Runtime: %g" % (end - start))

Training on 2000 batches:
batches 1 - 100, duration ~ 34.8869 sec
batches 101 - 200, duration ~ 33.1564 sec
